# M2.2 - Travailler avec des données climatiques grindtées

* Participe: * [** Computational Climate Science **] (https://github.com/openClimatescience/M2-Compuational-climate-science) | ** Leçon précédente ** | ** Next Leçon **

**Contenu:**

- [contraintes conceptuelles et informatiques] (# Conceptual and Computational-Constaintes)
- [une série temporelle de stockage d'eau terrestre (TWS)] (# A-Terrestrial-Water-Storage- (TWS) -Time-Series)
  - [Accès aux variables `XArray`] (# Accès aux variables de xarray)
- [Penser aux tableaux multidimensionnels] (# Thinking-Sur-Multi-Dimensional-Arrays)
  - [Mise à jour des systèmes de coordonnées] (# Mise à jour-coordonnées-systèmes)
- [Indexation d'un tableau multidimensionnel] (# indexation-a-multidimensional-array)
- [Aggrégation le long de l'axe d'un tableau multidimensionnel] (# Aggregating-Aong-the-axis-of-A-Multi-Dimensional-Array)
- [Rééchantillonnage d'une série temporelle] (# Rééchantillonnage d'une série-au moment)
  - [Gestion des données manquantes] (# manutention-missing-data)
- [Visualiser la variation dans l'espace] (# Visualisation-variation-over espace)
- [Résumé] (# Résumé)

## contraintes conceptuelles et informatiques

À mesure que la technologie informatique s'améliore et que davantage de jeux de données par satellite sont disponibles, la taille, la complexité et la fréquence des données climatiques augmentent rapidement. Cela peut entraîner deux problèmes pour les chercheurs et les gestionnaires de ressources qui cherchent à utiliser les données climatiques. Premièrement, il peut être difficile de conceptualiser la taille et l'échelle de certains ensembles de données climatiques, conduisant à des problèmes avec les données et la gestion de projet (Jain et al. 2022). Deuxièmement, il peut être difficile de traiter et d'analyser des ensembles de données climatiques qui sont parfois trop grands pour s'adapter à la mémoire de l'ordinateur ou trop complexes pour un calcul tractable.

Dans cette leçon, nous explorerons comment gérer certains de ces problèmes. Parce que nous apprenons, nous utiliserons un ensemble de données qui n'est pas aussi grand ou complexe que d'autres que nous pourrions rencontrer, mais cela devrait servir de bonne illustration des ensembles de données plus difficiles et comment les gérer.

In [ ]:
import xarray as xr
import earthaccess
import numpy as np
from matplotlib import pyplot

auth = earthaccess.login()

## Une série temporelle de stockage d'eau terrestre (TWS)

Pour acquérir plus d'expérience en travaillant avec les données climatiques grindtées, et en particulier une série chronologique de données climatiques grindées, nous utiliserons un ensemble de données de stockage d'eau terrestre (TWS) produit à l'aide de données à partir de deux missions par satellite: la mission de la récupération de la gravité et de l'expérience climatique (Grace) et son successeur, la mission de suivi Grace (Grace-Fo).

TWS comprend toute l'eau sur et sous la surface terrestre, sous forme de manteau neigeux, de ruisseaux de surface et de réservoirs et d'eau souterraine. [L'ensemble de données TWS avec lequel nous travaillerons à partir des missions Grace et Grace-Fo] (https://podaac.jpl.nasa.gov/dataset/tellus_grac-grfo_mascon_cri_grid_rl06.1_v3) a déjà été convertie en * anomalies mensuelles. *

& # x1F449; [En savoir plus sur les données Grace / Grace-Fo ici.] (Https://grace.jpl.nasa.gov/about/faq/)

Rappelons que les * anomalies * sont un moyen de représenter la variabilité autour d'une moyenne à long terme. Ils nous disent quelque chose sur la façon dont un point de données particulier (un jour, un mois ou une année particulier) se compare au fil du temps; Par exemple, est-ce une année particulièrement sèche?

#### & # x1f3af; Meilleure pratique

* Nous sommes sur le point de télécharger des données brutes! Vous savez quoi faire. * Créez un nouveau dossier appelé «Grace» dans le dossier `data_raw».

In [ ]:
results = earthaccess.search_data(short_name = 'TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3')
results

Cet ensemble de données a été préparé en tant que fichier NetCDF unique.

In [ ]:
earthaccess.download(results, 'data_raw/GRACE')

** Vous pouvez obtenir un message d'erreur lorsque vous essayez de télécharger cet ensemble de données en raison d'un problème avec les serveurs de la NASA. ** Si c'est le cas, notez qu'il existe une seule URL pour cet ensemble de données (inclus dans la sortie ci-dessus), ce que vous pouvez Utilisez pour télécharger les données directement. Il devrait être:

- [https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/tellus_grac-grfo_mascon_cri_grid_rl06.1_v3/grctellus.jpl.200204_202311.glo.rl06.1m (https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/tellus_grac-grfo_mascon_cri_grid_rl06.1_v3/grctellus.jpl.200204_202311.glo.rl06.1m (Cliquez pour télécharger)

In [ ]:
ds = xr.open_dataset('data_raw/GRACE/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc')
ds

Les données Grace / Grace-Fo représentent déjà * les anomalies *, c'est-à-dire un changement du stockage total de l'eau par rapport à une période de temps (ou "époque"). Quelle période est-ce? Nous pouvons le découvrir à partir des attributs de niveau ** de fichier ** de ce fichier NetCDF.

In [ ]:
ds.attrs['time_mean_removed']

Nous pouvons également le vérifier en examinant une série chronologique de la moyenne mondiale dans les anomalies TWS mensuelles. Pour la période 2004-2009, ces anomalies semblent être centrées sur zéro, ce que nous nous attendons, car la moyenne de cette période a été soustraite des données.

Nous voyons également une tendance à long terme à la baisse. Qu'est-ce qui pourrait expliquer cela? La perte mondiale de glace terrestre est probablement le principal facteur qui stimule cette perte. Le rapport d'évaluation le plus élevé (AR6) du panel intergouvernemental sur le changement climatique (GIEC) estime que la fonte des glaciaires et des glaces depuis 2006 et 2018 représente près de 2 centimètres d'élévation du niveau de la mer (GIEC 2021, chapitre 9), ce qui est cohérent avec ce que nous voyons ici (l'axe vertical de notre parcelle est également en centimètres). Le réchauffement de l'atmosphère terrestre joue également un rôle, car l'air plus chaud peut contenir plus de vapeur d'eau.

In [ ]:
ds['lwe_thickness'].mean(['lon','lat']).plot()

L'anomalie de la série chronologique vers 2017-2018 est due au fait que la mission de Grace s'est terminée en octobre 2017 et que Grace-Fo n'a été lancée qu'en mai 2018; Par conséquent, il n'y a pas de valeurs de données pour ces mois.

Nous pouvons regarder plus d'une série chronologique: il s'agit également d'un ensemble de données spatial. Jetons un coup d'œil aux anomalies mondiales TWS pour le premier rendez-vous.

In [ ]:
ds['lwe_thickness'][0].plot(vmin = -200, vmax = 200, cmap = 'RdBu')

Dans la parcelle ci-dessus, qui représente l'anomalie mensuelle TWS pour avril 2002, les couleurs fraîches montrent une anomalie TWS positive (c'est-à-dire plus d'eau que d'habitude pour avril) et des couleurs plus chaudes montrent une anomalie TWS négative (c'est-à-dire moins d'eau que d'habitude pour avril) .

### Accès aux variables `xarray»

Notez que les coordonnées et les variables de données dans les ensembles de données «XArray» sont accessibles de deux manières différentes.

- Le nom de la coordonnée ou de la variable utilisée comme clé de dictionnaire: `ds ['lwe_thickness']` ou `ds ['temps']`
- Le nom de la coordonnée ou de la variable utilisée comme attribut: `ds.lwe_thickness` ou` ds.time`

Quelle que soit la syntaxe que vous utilisez, le résultat est le même.

---

## Penser aux tableaux multidimensionnels

** Rouvons notre ensemble de données à l'aide de `Open_MFDATASET ()`. ** Rappelons que nous utilisons le `Open_MFDATASET ()` de `XArray` chaque fois que nous voulons ouvrir plusieurs fichiers comme s'il s'agissait d'un seul ensemble de données. Ici, nous n'avons qu'un seul fichier, mais nous prétendons qu'il s'agit de plusieurs fichiers car il contient une image d'anomalie TWS globale pour plusieurs dates.

In [ ]:
ds = xr.open_mfdataset('data_raw/GRACE/GRCTellus.JPL.200204_202311.GLO.RL06.1M.MSCNv03CRI.nc')

Comme nous l'avons vu, les fichiers NetCDF4 et HDF5 peuvent être représentés sous forme de jeux de données `xArray».

In [ ]:
type(ds)

Un ensemble de données peut contenir plus d'une variable, chacun représenté comme un tableau multidimensionnel.

Il existe différentes façons de représenter des tableaux dans Python. Lorsque nous utilisons «XArray», les variables sont représentées comme DataArrays.

In [ ]:
type(ds['lwe_thickness'])

Le `XARray 'DataArray est généralement juste un type spécial de tableau Numpy. Mais lorsque nous utilisons `xarray.open_mfdataset ()` Nous obtenons un nouveau type de tableau.

In [ ]:
type(ds['lwe_thickness'].data)

** La bibliothèque `Dask` est automatiquement utilisée pour représenter notre tableau multidimensionnel sous-jacent. plusieurs fichiers.

** Mais en utilisant `xarray.open_mfdataset ()` pour accumuler plusieurs fichiers ensemble dans une série temporelle, nous avons également acquis une nouvelle façon de penser à nos données. ** Chaque fichier individuel représentait essentiellement une image 2D: l'anomalie TWS sur une grille de latitude-longitude. Avec plusieurs dates d'image empilées, nous obtenons un cube de données 3D **, ** avec X (longitude), y (latitude) et des axes temporels. Les ensembles de données «XArray» peuvent afficher une illustration utile lorsque nous les utilisons dans un cahier Jupyter.

In [ ]:
ds['lwe_thickness'].data

Lors de l'analyse des cubes de données, nous essayons généralement de répondre à l'une de ces questions:

1. Comment une variable climatique dans un ou plusieurs emplacements varie-t-elle au fil du temps?
1. Comment une variable climatique à un certain moment varie-t-elle sur l'espace?
1. Comment la variabilité climatique (variation dans le temps) se compare-t-elle dans l'espace?

Les deux premières questions ne nécessitent pas un cube de données pour répondre, mais nous devons souvent fusionner différents ensembles de données pour y répondre. Nous pourrions nous considérer chanceux de commencer par un cube de données, dans ce cas, car nous avons toutes les données en un seul endroit. Ces questions nous obligent à sous-ensemble ou * index * un tableau multidimensionnel.

La troisième question ne peut être répondue que si nous commençons par un cube de données; C'est une question qui nous oblige à agréger ou à s'effondrer * un axe de notre cube de données.

### Mise à jour des systèmes de coordonnées

Rappelez-vous que les ensembles de données «xArray» ont des systèmes de coordonnées, qui codent les informations spatiales des données.

In [ ]:
ds.coords

Nous pouvons voir que nos données sont sur une grille de latitude à demi-degré-longitude; c'est-à-dire que chaque pixel est de 0,5 degrés par 0,5 degrés.

Vous avez peut-être remarqué que si les latitudes, ou les valeurs «lat», s'étendent à -90 degrés à 90 degrés, les valeurs de longitude ou «lon», s'étendent 0 à 360 degrés. Nous avons l'habitude de parler de longitudes qui s'étendent sur -180 degrés (ouest) de longitude à 180 degrés (est) de longitude. Comment pouvons-nous résoudre ce problème?

#### & # x1F6A9; <span style = "Color: Red"> Faites attention </DID>

** Changeons nos valeurs de «lon» afin qu'elles soient plus faciles à travailler. Maintenant 180.

In [ ]:
# Run this only once!
ds.coords['lon'] = ds.coords['lon'] - 180

In [ ]:
ds.coords

** Les coordonnées sont comme des étiquettes pour les points de données de notre cube de données, tandis que les dimensions sont les axes du cube de données. ** Nous pouvons appliquer plusieurs étiquettes à la même dimension. Par exemple, nous avons à la fois une dimension nommée «Time» et un ensemble de coordonnées nommé «Time». La coordonnée du temps "décrit le moment précis dans le temps jusqu'à la nanoseconde (NS), mais nous pourrions également étiqueter les points de temps de notre ensemble de données d'ici l'année, le mois, la saison ou le jour.

** Ajoutons une nouvelle coordonnée de «l'année» à notre ensemble de données, afin que nous puissions identifier les points de données individuels par une année courante. Ensuite, nous utilisons [la méthode `Assign_Coords ()`] (https://docs.xarray.dev/en/stable/generated/xarray.dataset.assign_coords.html) d'un `xarray.dataset` le long d'une dimension existante.

In [ ]:
# Get a list of all the dates from the "time" coordinate
dates = ds.coords['time'].values

Jetez un œil à ce que contient les «dates».

`` Python
dates
`` '

In [ ]:
# Convert each "datetime64[ns]" object to a 4-digit year
years_list = []
for each in dates:
    years_list.append(int(str(each)[0:4]))

Jetez un œil à ce que contient `` Years_list '.

`` Python
années_liste
`` '

In [ ]:
# Create a new coordinate ("year") along an existing dimension ("time")
ds = ds.assign_coords(year = ('time', years_list))
ds

---

## Indexation d'un tableau multidimensionnel

Nous pouvons couper notre cube de données plusieurs façons différentes de répondre à différentes questions. Par exemple, à quoi ressemblent les 20 dernières années d'anomalies TWS dans l'ouest des États-Unis, étant donné que la sécheresse multi-décadale a ressentie que la région a ressentie?

Commençons par regarder le pixel qui contient Sacramento, ca. Trancher notre tableau à ce seul pixel équivaut à prendre une mince étable le long de l'axe de temps; Comme illustré ci-dessous, nous nous retrouvons avec ce qui est essentiellement un tableau 1D de 220 valeurs (pour 220 mois de données).

In [ ]:
ds['lwe_thickness'].sel(lon = -121.75, lat = 38.25).data

Cela équivaut à trancher notre tableau le long des axes x et y (longitude et latitude), comme nous pouvons le voir ci-dessous.

In [ ]:
# Three dimensions: (time, lat, lon)
ds['lwe_thickness'].shape

In [ ]:
# Take the first pixel along the "lat" dimension and the first pixel along the "lon" dimension
ds['lwe_thickness'][:,0,0].data

Les données pourraient être représentées comme une série chronologique, car nous avons une valeur au fil du temps.

In [ ]:
ds['lwe_thickness'].sel(lon = -121.75, lat = 38.25).plot(figsize = (12, 6))

** Si nous voulions regarder plusieurs pixels représentant une région d'intérêt, nous tranchons toujours notre tableau le long de l'axe temporel, mais nous finissons par un cube de données plus petit. **

Rappelez-vous que la fonction «Slice ()» intégrée peut être utilisée en combinaison avec la méthode «sel ()» de `xarray» pour sélectionner une région d'intérêt. Dans ce cas, notre région mesure 10 pixels de 10 pixels de haut.

In [ ]:
west_us = ds['lwe_thickness'].sel(lon = slice(-124, -114), lat = slice(32, 42))
west_us.data

Il n'y a pas de moyen facile de visualiser les valeurs d'un cube de données 3D, donc lorsque nous demandons à «XArray» de tracer les données, cela nous montre simplement un histogramme, en mettant essentiellement toutes les valeurs du cube de données ensemble.

In [ ]:
west_us.plot()

Bien sûr, nous pouvons toujours tracer un seul pas de temps en sous-ensemble l'ensemble de données.

In [ ]:
west_us.sel(time = '2003-01-01', method = 'nearest').plot()

D'après l'apparence de cette image (et si nous examinons les valeurs de tableau sous-jacentes), nous pouvons déduire que si cet ensemble de données de latitude-longitude a une résolution spatiale de 0,5 degrés, les valeurs sont répétées dans certaines zones parce que [la résolution spatiale réelle de la Les données sont plus proches de 300 km] (https://grace.jpl.nasa.gov/about/faq/).

---

## Aggrégation le long de l'axe d'un tableau multidimensionnel

Comme l'histogramme ci-dessus le suggère, nous avons besoin d'un moyen de transformer notre cube de données afin de mieux visualiser les données et de répondre à certaines de nos questions.

** Quelle est l'anomalie TWS moyenne dans notre zone d'étude dans chaque mois? ** On peut répondre en moyenne sur le domaine spatial. Nous pouvons visualiser cela comme une moyenne de chaque tranche de 10 par 10 pixels dans notre cube de données, pour chaque pas de temps mensuel.

In [ ]:
west_us.data

Si nous spécifions que nous voulons en moyenne l'un de nos axes spatiaux, nous passons d'un tableau 3D à un tableau 2D; c'est-à-dire, ** Nous avons effondré l'un de nos axes. **

In [ ]:
west_us.mean('lat').data

Plus précisément, nous avons perdu la dimension `" lat ", comme nous pouvons le voir ci-dessous.

In [ ]:
west_us.mean('lat').dims

Montons en moyenne plus de deux de nos axes spatiaux: en moyenne sur la latitude * et * la longitude. Cela signifie que nous passons d'un tableau 3D à un tableau 1D; Bien qu'il soit représenté, ci-dessous, en tant que réseau 2D, nous avons un axe trivial de longueur un (1), nous n'avons donc vraiment qu'une séquence 1D de 220 valeurs.

In [ ]:
west_us.mean(['lat', 'lon']).data

Et comme nous avons une série chronologique 1D d'anomalies TWS, nous pouvons les visualiser comme un tracé de ligne. De ce complot, nous pouvons voir que la sécheresse multi-décennale dans l'ouest des États-Unis s'est aggravée ces dernières années.

In [ ]:
west_us.mean(['lat', 'lon']).plot(figsize = (12, 6))

---

## rééchantillonnant une série chronologique

Une autre tâche d'analyse des données courante que nous pourrions vouloir effectuer avec les données de séries chronologiques consiste à réagir les données; Par exemple, pouvons-nous calculer l'anomalie moyenne * annuelle * TWS, pour mieux caractériser les années humides et sèches?

In [ ]:
west_us.data

[La méthode `resample ()` d'une `xarray.dataarray`] (https://docs.xarray.dev/en/stable/generated/xarray.dataarray.onleample.html) est le meilleur moyen d'agréger les séries temporelles données.

In [ ]:
west_us_annual = west_us.resample(time = 'YS').mean()
west_us_annual.data

La syntaxe `temps = 'ys' ', ci-dessus, indique que le * début de l'année * (` ys »abrégé) doit être utilisé comme fréquence de rééchantillonnage; c'est-à-dire calculer la moyenne pour chaque année. Cette syntaxe provient de la bibliothèque «Pandas». 

& # x1F449; [** En savoir plus sur `resample ()` et rééchantillonnage des fréquences ici. **] (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

Nous devons également utiliser `Resemple () 'pour certains calculs que nous effectuons, tels que le calcul des tendances temporelles, où les unités d'une tendance sont libellées par l'unité de temps (par exemple," par jour "ou" par an "). En effet, `COARSEN () 'ne change pas l'unité de temps; Dans ce cas, cela ne change pas les unités de temps de jours à des années.

Ainsi, en général, avec les axes de séries chronologiques, il est préférable d'utiliser la méthode `Reslempl () 'de DataArray.

Mais notez que lorsque nous rééchantillons le long d'une dimension donnée, toutes les étiquettes de coordonnées ne sont pas conservées. Dans ce cas, `xarray` ne sait pas comment attribuer une étiquette« année »aux coordonnées rééchantillonnées.

In [ ]:
# We lost the "year" coordinate
west_us_annual.coords

#### & # x1F6A9; <span style = "Color: Red"> Faites attention </DID>

Mais dans ce cas particulier, puisque nous sommes intéressés à agréger par année, de toute façon, nous n'avons pas à utiliser «resample ()»; Nous pouvons utiliser `GroupBy () 'pour indiquer que nous voulons prendre la moyenne dans chaque étiquette de coordonnées« année ».

In [ ]:
west_us_annual = west_us.groupby('year').mean()
west_us_annual

### Gestion des données manquantes

Avec des DATSET basés sur des satellites comme cet ensemble de données TWS, il y a toujours une possibilité de données manquantes. Par exemple, il y a un écart en 2017 et 2018 à la fin de la mission Grace, avant le lancement du satellite Grace-Fo. Il manque également des mois en 2002, avant le lancement de Grace en mars, et les données Grace-Fo 2023 n'incluent pas décembre 2023, car elles sont toujours en cours de traitement.

& # x1F449; ** Notez que, parce que vous accédez à ces données du cloud de la NASA, vous pouvez constater que des données plus récentes (avec moins de mois manquantes) sont disponibles! **

Ci-dessous, nous comptons combien de mois sont disponibles chaque année.

In [ ]:
counts = ds.groupby('year').count()

# Get a value (the count) in each year for an arbitrary pixel, skipping the first year (2002)
counts['lwe_thickness']

** Cependant, nous obtenons un compte des mois dans chaque pixel! ** Nous voulons juste le décompte pour un pixel (arbitraire) car, avec ce produit TWS modélisé, la disponibilité des données est la même pour chaque pixel.

In [ ]:
# Get the count at an arbitrary pixel: the top-left (0,0) corner
counts_by_year = counts['lwe_thickness'][:,0,0].values
counts_by_year

Il est évident qui nous manque plusieurs mois de données, en particulier en 2017 et 2018 (seulement 5 mois disponibles chaque année) entre les missions Grace et Grace-Fo.

Nous devons d'abord nous assurer que nous avons au moins quelques données chaque année. Ci-dessous, nous pouvons voir que notre axe temporel a 21 éléments (21 ans). Cela s'aligne avec nos dénombrements mensuels lorsque nous ignorons la première année (2002).

In [ ]:
west_us_annual.shape

In [ ]:
counts_by_year.size

** Imposons une règle: nous avons besoin d'au moins 9 mois de données chaque année pour une moyenne annuelle fiable. **

In [ ]:
mask = counts_by_year < 9
mask

In [ ]:
west_us_annual[mask] = np.nan

Il convient de rappeler qu'un correctif connexe pour ce problème, si tous les points de données valides étaient consécutifs, serait de prendre une tranche:

In [ ]:
# A related fix, if the valid data points were all consecutive
west_us_annual = west_us_annual.sel(year = slice(2003, 2023))
west_us_annual.year

Les intervalles de temps d'agrégation ne sont qu'une des nombreuses choses intéressantes qu'un `XArray.DataArray 'peut faire! [Consultez la documentation `xarray` pour en savoir plus.] (Https://docs.xarray.dev/en/stable/generated/xarray.dataarray.html)

In [ ]:
west_us_annual.mean(['lat','lon']).plot()

Nous pouvons observer une tendance générale à la baisse dans les données, ce qui est conforme à la sécheresse depuis des décennies que la région a ressentie (Liu et al. 2022).

---

## Visualiser la variation dans l'espace

Dans l'exemple précédent, nous avons rééchantillonné nos données sur l'axe temporel (au fil du temps). Mais notre ensemble de données a également des coordonnées spatiales! Nous voulons souvent visualiser comment la variation temporelle se compare entre différents emplacements. Il s'agit simplement de rééchantillonner notre cube de données d'une manière différente.

In [ ]:
west_us_annual.data

Nous pourrions calculer une moyenne interannuelle simple. Cela effondre notre cube de données à deux dimensions spatiales, la latitude et la longitude.

In [ ]:
west_us_annual.mean('year').data

** Cependant, ce serait beaucoup plus intéressant si nous pouvions calculer les tendances au niveau des pixels dans les anomalies TWS annuelles. **

Parce que nous travaillons avec `xarray ', une bibliothèque Python très efficace et mature pour travailler avec des tableaux multidimensionnels, c'est une bonne idée de vérifier si la fonctionnalité que nous voulons est déjà intégrée à la classe` xarray.dataarray`. En effet, il existe deux fonctions qui pourraient aider ici:

- `dataArray.curveFit ()` [(documentation)] (https://docs.xarray.dev/en/stable/generated/xarray.dataarray.curvefit.html#xarray.dataarray.curvefit)
- `dataarray.polyfit ()` [(documentation)] (https://docs.xarray.dev/en/stable/generated/xarray.dataarray.polyfit.html#xarray.dataarray.polyfit)

L'un ou l'autre peut être utilisé pour ajuster une fonction à un ensemble de points de données. Cependant, nous voulons utiliser `Polyfit () 'aujourd'hui parce que nous montons une fonction simple (une tendance linéaire) qui devrait avoir [une solution de forme fermée] (https://en.wikipedia.org/wiki/cosé-forme_expression ). Un polynôme "degré 1" est une ligne droite car, dans la fonction impliquée, $ y (x) = mx + b $, $ x $ est augmenté à la puissance de 1: $ x ^ 1 $.

In [ ]:
fit = west_us_annual.polyfit('year', deg = 1)
fit

In [ ]:
fit['polyfit_coefficients']

** Nous avons obtenu un autre cube de données! ** Ce cube de données a un axe appelé ** degré ** avec deux (2) éléments. Bien que nous ayons demandé un ajustement polynomial à 1 degré, rappelez-vous que nous avons besoin de deux nombres pour décrire une ligne: l'ordonnée Y et la pente.

#### & # x1F6A9; <span style = "Color: Red"> Faites attention </DID>

Une autre chose à noter sur notre cube de données est que nous avons une ligne de tendance (interception et pente) pour chacun des 400 pixels. Ces 400 régressions linéaires semblaient très rapides! 

** En fait, nous n'avons pas encore calculé les régressions linéaires. En effet, nous avons chargé les données TWS dans la mémoire de l'ordinateur en utilisant `xr.open_mfdataset ()`. Par conséquent, «XArray» ne fait aucun calcul jusqu'à ce que nous le demandons explicitement.

Alors, que s'est-il passé lorsque nous avons tapé ce qui suit?

`` Python
fit = west_us_annual.polyfit ('année', deg = 1)
`` '

Ce qui s'est passé, c'est que nous avons obtenu une ** représentation ** du calcul que nous voulions que «XArray» fonctionne. Nous avons appelé cette représentation «Fit» et cela ressemble à un cube de données parce que c'est à quoi ressemblera le calcul lorsqu'il sera effectué.

** Lorsque nous sommes prêts à charger des données ou à calculer les résultats, [nous devons appeler la méthode `calcul ()`.] (Https://docs.xarray.dev/en/stable/generated/xarray.dataarray. .html) **

In [ ]:
# Get a representation of the computation we want to do
fit = west_us_annual.polyfit('year', deg = 1)

# Actually run the computation
results = fit.compute()

# Look at the coefficients
results['polyfit_coefficients']

En effet, `xArray` utilise ** Évaluation paresseuse; ** Les données ne sont chargées que dans la mémoire et traitées lorsqu'elle est nécessaire. Cela aide à éviter les opérations peu sûres et souvent lentes, lues et écrites.

** Il existe deux façons de s'assurer que les calculs sur les ensembles de données et les données de données `xarray 's'exécutent réellement lorsque vous le souhaitez: **

- Appelez la méthode `calcul () ', comme dans l'exemple ci-dessus.
- ou, essayez d'accéder à la propriété `` valeurs ', comme dans l'exemple ci-dessous.

`` Python
ajuster ['polyfit_coefficients'].
`` '

---

## Résumé

Voici les principaux outils et techniques Python à retenir de cette leçon:

- Vous pouvez utiliser `xarray.open_mfdataset ()` pour ouvrir plusieurs fichiers en tant que jeu de données `XArray». Par exemple, vous pourriez écrire
  `data = xarray.open_mfdataset (" prepip_in_year_y * .nc ")` Pour ouvrir une série de fichiers de données annuels de précipitations.
  
- Les attributs (ou métadonnées) de tout fichier ouvert dans `xArray` sont accessibles via:` data.attrs`
  
- Les ensembles de données `xArray` sont également parfois appelés ** Cubes de données. ** Vous pouvez agréger un ou plusieurs axes ** ** (ou" Dimensions ") du cube de données à l'aide de méthodes comme` Mean () '; Par exemple, en moyenne sur les deux dimensions spatiales de la longitude et de la latitude: `données ['variable']. Mean (['lon', 'lat'])`.
  
- Pour trancher un cube de données le long d'un ou plusieurs axes, utilisez la méthode `sel ()`, par exemple: `data ['variable']. Sel (lon = -121,75, lat = 38.25)`.

- La méthode `Reslemple ()` d'une `xarray.dataarray` ou` xarray.dataset »doit être utilisée pour agréger les données de séries chronologiques. Par exemple, pour agréger les données horaires en pas de temps quotidiens ou des données mensuelles en pas de temps annuels. [En savoir plus sur le rééchantillonnage des fréquences ici.] (Https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

- `XArray` utilise ** Évaluation paresseuse ** pour éviter la lecture, l'écriture ou le calcul des données jusqu'à la dernière minute; Pour forcer `xarray` à charger ou évaluer les données, vous pouvez appeler [la méthode` calcul () `] (https://docs.xarray.dev/en/stable/generated/xarray.dataarray.compute.html) ou accès L'attribut `valeurs` d'un` xArray.dataArray`.

---

### Plus de ressources

- [Computation sur les ensembles de données et DataArrays `XArray`] (https://docs.xarray.dev/en/stable/user-guide/comportation.html)

### références

GIEC. * Changement climatique 2021: la base des sciences physiques. Contribution du groupe de travail I au sixième rapport d'évaluation du panel intergouvernemental sur le changement climatique. * 2021. Cambridge University Press, Cambridge, Royaume-Uni et New York, NY, États-Unis, 2391 pp. DOI: 10.1017 / 9781009157896

Jain S, Mindlin J, Koren G, Gulizia C, Steadman C, Langendijk GS, Osman M, Abid MA, Rao Y, Rabanal V. 2022. [Sommes-nous à risque de perdre la génération actuelle de chercheurs climatiques à la science des données?] (https://doi.org/10.1029/2022av000676) * Agu avance. *

Liu, P.-W., J. S. Famiglietti, A. J. Purdy, K. H. Adams, A. L. McEvoy, J. T. Reager, R. Bindlish, D. N. Wiese, C. H. David et M. Rodell. 2022. [Déplétion des eaux souterraines dans la vallée centrale de Californie accélère pendant MegadRought.] (Https://www.nature.com/articles/s41467-022-35582-x) Nature Communications 13 (1): 7825.

`xArray.DataArray` (XArray Documentation). [https://docs.xarray.dev/en/stable/generated/xarray.dataarray.html , 2024.